## Use PromptTemplate to create a template for a string prompt
By default, PromptTemplate use Python's str.format syntax for templating

In [58]:
import os
from dotenv import load_dotenv
import openai
#
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, AIMessagePromptTemplate
from langchain_core.messages import SystemMessage

In [59]:
# Setup model
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key

In [60]:
model = OpenAI(openai_api_key=api_key,model="gpt-3.5-turbo-instruct",temperature=0.5) #gpt-4-0125-preview

### A simple string based Prompt formatting

In [61]:
prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}")
prompt = prompt_template.format(adjective="stupid",content="Vietnamese")
print("-----------------")
print(prompt_template)
print("-----------------")
print(prompt)
response = model.invoke(prompt)
print("-----------------")
print(response)

-----------------
input_variables=['adjective', 'content'] template='Tell me a {adjective} joke about {content}'
-----------------
Tell me a stupid joke about Vietnamese
-----------------


Why did the Vietnamese man go to the doctor?

Because he was having a pho-king headache!


### ChatPromptTemplate: The prompt to chat model is a list of chat message
Each chat message is associated with content and an additional parameter called role.

For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human, or a system role.

(Mỗi tin nhắn trò chuyện được liên kết với nội dung và một tham số bổ sung gọi là vai trò.

Ví dụ: trong API hoàn thành trò chuyện OpenAI, tin nhắn trò chuyện có thể được liên kết với trợ lý AI, con người hoặc vai trò hệ thống.)

In [62]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI bot. Your name is {name}"),
        ("human","Hello, how are you doing?"),
        ("ai","I'm doing well, thanks!"),
        ("human","{user_input}"),
    ]
)
prompt = chat_template.format_messages(name="Bob",
                                       user_input="What is you name and what are you good at?"
                                      )
print("-----------------")
print(chat_template)
print("-----------------")
print(prompt)
response = model.invoke(prompt)
print("-----------------")
print(response)

-----------------
input_variables=['name', 'user_input'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='You are a helpful AI bot. Your name is {name}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Hello, how are you doing?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="I'm doing well, thanks!")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))]
-----------------
[SystemMessage(content='You are a helpful AI bot. Your name is Bob'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is you name and what are you good at?')]
-----------------

AI: My name is Bob and I am good at assisting with tasks, providing information, and learning new things. Is there something specific you need help with?


### Various ways of formatting System/Human/AI prompts

In [64]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to sound more beat" # Bạn là một trợ lý hữu ích giúp viết lại văn bản của người dùng để nghe có nhịp điệu hơn
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
        AIMessagePromptTemplate.from_template("Give an answer in English and then translate the answer to Vietnamese")
    ]
)
prompt = chat_template.format_messages(text="I don't like eating tasty things") #Tôi không thích ăn đồ ngon.
print("-----------------")
print(chat_template)
print("-----------------")
print(prompt)
response = model.invoke(prompt)
print("-----------------")
print(response)

-----------------
input_variables=['text'] messages=[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more beat"), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Give an answer in English and then translate the answer to Vietnamese'))]
-----------------
[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more beat"), HumanMessage(content="I don't like eating tasty things"), AIMessage(content='Give an answer in English and then translate the answer to Vietnamese')]
-----------------


I'm not a fan of indulging in delicious treats.
Tôi không thích ăn những thứ ngon lành.


### Providing a Context along with the Prompt

In [65]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """
# Context: Mô hình ngôn ngữ lớn (LLM) là những mô hình mới nhất được sử dụng trong NLP.
# Hiệu suất vượt trội của chúng so với các mô hình nhỏ hơn đã khiến chúng cực kỳ hữu ích cho các nhà phát triển xây dựng các ứng dụng hỗ trợ NLP. 
# Bạn có thể truy cập các mô hình này thông qua thư viện `transformers` của Hugging Face, qua OpenAI bằng thư viện `openai` và qua Cohere bằng thư viện `cohere`.

In [66]:
print(model.invoke(prompt))

Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library.
